# CYPLAN255
### Urban Informatics and Visualization

# Lecture 15 -- Geospatial Data Analysis
******
March 20, 2024

<img src="https://s3-eu-west-1.amazonaws.com/ngs-mw-prod/9/97/11799/11799.jpg" width=500 align='right' title='Alfred G Buckham, Aerial View of Edinburgh (c. 1920)'>

# Agenda
1. Announcements
2. Final projects
3. An example
4. More Geopandas
5. Summary
6. For next time
7. Questions


# 1. Announcements

- Final project groups [sign-up sheet](https://docs.google.com/spreadsheets/d/1aTUUA7cqtUg-5jWVrz2XrmS25SR7w-Yz6pYLzZySKOA/edit?usp=sharing)

# 2. Final Projects

In [ ]:
from IPython.display import IFrame
IFrame("../misc/final_project_description.pdf", width="100%", height=700)

# 3. An example

**Historical Redlining Is Associated with Present-Day Air Pollution Disparities in U.S. Cities**[<sup id="fn1-back">1</sup>](#fn1)

<img src="https://pubs.acs.org/cms/10.1021/acs.estlett.1c01012/asset/images/large/ez1c01012_0003.jpeg" width=700>

Two open data sets:
  - Census Block-level air pollution data available from [CACES](https://www.caces.us/data)
  - Redlining data available from https://dsl.richmond.edu/panorama/redlining/

[<sup id="fn1">1</sup>](#fn1-back) Lane, H. M., Morello-Frosch, R., Marshall, J. D., & Apte, J. S. (n.d.).  Environmental Science & Technology Letters, 0(0), null. https://doi.org/10.1021/acs.estlett.1c01012

In [ ]:
%%html
<iframe src="https://dsl.richmond.edu/panorama/redlining/" width=100% height=700>

**Question**

- Correlation or causation?

- How might we inch our way towards causation?

# 4. More GeoPandas - Berkeley Edition

For this exercise we're going to download the Census Block geometries for the City of Berkeley:

In [ ]:
import geopandas as gpd
blocks = gpd.read_file("https://data.cityofberkeley.info/api/geospatial/caxd-afre?method=export&format=Shapefile")

In [ ]:
blocks.plot(color='none', edgecolor='gray', linewidth=.2, figsize=(14,10))

## 4.1. Our First Geoprocessing Step: Dissolve

Let's say we want to create a census tract dataset from the census block one, which we can see from inspecting the data above, contains a column called tractce10, for 2010 census tracts.  We can verify it by looking more closely at the geoid10 field, which appears to be a FIPS code.

Geoprocessing steps like dissolve do geometric processing on the geometry column of a GeoSeries or GeoDataFrame (which contains a GeoSeries column).

In [ ]:
blocks.head()

In [ ]:
tracts = blocks.dissolve(by='tractce10')
tracts.plot(figsize=(10,8))

## 4.2. Setting the crs of GeoDataFrames

Let's see what the crs is for the newly created dataset, and what it's type is.

In [ ]:
print(tracts.crs)
print(type(tracts))

OK, so dissolve created a new GeoDataFrame, and assigned it the same crs as the original GeoDataFrame. But just to be sure you know how to do this in the event you load a file and the crs is undefined, we will set the crs in order to do other spatial operations on the tract dataset. 

In [ ]:
tracts.crs = {'init' :'epsg:4326'}
# or alternatively:
tracts.crs = blocks.crs
print(tracts.crs)

## 4.3. Loading 311 Cases for Grafitti and Vandalism as Point Data

Let's load 311 data for graffiti and vandalism, dropping rows that have missing data (a good fraction of the data seem to be missing latitude and longitude)

In [ ]:
import pandas as pd
vandalism = pd.read_json("https://data.cityofberkeley.info/resource/bscu-qpbu.json?request_category=Graffiti%20and%20Vandalism&$limit=20000")

In [ ]:
vandalism.head()

We loaded a csv into a standard pandas DataFrame.  But it contains Latitude, Longitude columns, so with a couple of additional steps we can turn this into a GeoDataFrame, and set its crs.

Study this example carefully. It is very common to encounter spatial data in a tabular form that you'll want to quickly turn into a geometry data types.  Note the use of the built-in `zip()` function within a list comprehension to pull the Latitude and Longitude columns together to define each Point geometry.

In [ ]:
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(vandalism.longitude, vandalism.latitude)]
geovandalism = gpd.GeoDataFrame(vandalism, crs='epsg:4326', geometry=geometry)
geovandalism.head()

## 4.4. Mapping With Layers

Let's see what we have:

In [ ]:
geovandalism.plot(markersize=3);

A bunch of points mapped in isolation is not very informative, since it lacks context.  Let's add the points to the block base to add visual context. Note the use of Matplotlib syntax to set the parameters we want for the figure.

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(figsize=(18,10))
blocks.plot(color='white', edgecolor='black', linewidth=.1, ax=ax)
geovandalism.plot(markersize=4, ax=ax)
ax.set_title('Vandalism in Berkeley')
ax.set_axis_off()

## 4.5. Projecting to a Different crs

So far our data has been in world coordinates (sometimes referred to as geographic cordinates). This works fine for some purposes like generating basic maps, other than the fact that the maps can appear distorted at different scales because we are squashing a spherical coordinate system onto a flat image. But there is a more fundamental issue: distances measured on a spherical coordinate system will be incorrect for point to point distance measurements we often want to do in urban data analysis.  Since we will be doing some spatial analysis on these data, we need to project them to a coordinate system that allows more meaningful measurements of distance. We'll use a standard Bay Area CRS for this purpose: EPSG:26910. You can find out more about this CRS [here](http://spatialreference.org/ref/sr-org/6787/)

In [ ]:
geovandalism_proj = geovandalism.to_crs("epsg:26910")
blocks_proj = blocks.to_crs("epsg:26910")
tracts_proj = tracts.to_crs("epsg:26910")
tracts_proj.crs

Let's check the bounds for the whole dataset to see what kind of coordinates we're working with:

In [ ]:
bounds = tracts_proj.total_bounds
print(bounds)

## 4.6. Spatial Join: Intersect

A very common geoprocessing operation is to do a **point-in-polygon** assignment using an intersection of the geometries of points and polygons.  It is like a normal merge, where the two datasets do not have a common key to merge on, but instead these tables have coordinates that enable geometric processing to find matching rows.

GeoPandas makes this pretty easy using a spatial join, or `gpd.sjoin()` function with the `intersects` operation argument.

In [ ]:
geovandalism_proj_blocks = gpd.sjoin(geovandalism_proj, blocks_proj, how="inner", op='intersects')
geovandalism_proj_blocks.head()

Now we know the Census Block ID of each record in the vandalism data!

## 4.7. Aggregating data using Groupby

Let's say we want to look at the incidence of vandalism events at some level of geography like neighborhood or census tract rather than just looking at the raw data.  We can use groupby operations to get the counts of events within each geographic area, then merge it on to a Geodataframe to visualize it.

We use a reset_index to create a unique index and make the groupby column become a column in the resulting dataframe instead of the index. That makes the merge a bit clearer.

In [ ]:
tract_v = geovandalism_proj_blocks.groupby('tractce10')['case_id'].count().to_frame(name='total_vandalism').reset_index()
tract_v.head()

In [ ]:
tracts2 = pd.merge(tracts_proj,tract_v, left_index=True, right_on='tractce10')
tracts2.head()

## 4.8. Choropleth Maps

One of the more obvious things you might want to do with spatial data is visualize the variations in it spatially.  Besides plotting the individual points to see their pattern, we can generate a choropleth, or thematic, map by census tract.

In [ ]:
tracts2.plot(column='total_vandalism', figsize=(14,10), legend=True)

The default colormap is not very useful in this case. We can select a better colormap with the `cmap` param to  make our map more easily interpretable:

In [ ]:
tracts2.plot(column='total_vandalism', cmap='YlGn', figsize=(14,10), legend=True)

Now might be a good opportunity to spend a few minutes experimenting with different colormaps. A good place to start would be here: http://matplotlib.org/users/colormaps.html

Which kinds of colormaps seem to work best for this data?

## 4.9. The Modifiable Areal Unit Problem (MAUP)

One common issue when working with geospatial data is known as the Modifiable Areal Unit Problem (MAUP)

<center><img src="https://mikejohnson51.github.io/spds/lec-img/15-maup.png" width=600></center>

This comes up quite frequently when working with choropleth maps, since polygons with larger areas tend to show up more prominently than might be appropriate. This happens for two reasons:
  1. Large areas occupy more visual space on the map
  2. Larger area == more observations 

We'll now investigate a few ways for dealing with these issues

### 4.9.1. Use Equal-Area Bins
Perhaps the simplest approach would have been to just leave the Census Tracts out of it for the purpose of data viz

In [ ]:
# code won't work if we have any empty geometries
geovandalism_proj_blocks = geovandalism_proj_blocks[~geovandalism_proj_blocks.is_empty]

# plot
fig, ax = plt.subplots(figsize=(15,15))
g = blocks_proj.plot(color='none', edgecolor='k', linewidth=.25, ax=ax)
hb = ax.hexbin(x=geovandalism_proj_blocks['geometry'].x, y=geovandalism_proj_blocks['geometry'].y, gridsize=20, mincnt=1)
fig.colorbar(hb, shrink=.75)

### 4.9.2. Normalize by Area

We often want to normalize our data by area to compute a density to offset this. Geopandas gives us access to an attribute of the geometry of polygons to get the areas, so this is fairly straightforward to do.

But first, check to see what units our projection is in so we can convert to a known area size, like square miles.

In [ ]:
tracts2.crs

In [ ]:
tracts2['area_sqmi'] = tracts2.area / 3.861e-7
tracts2['van_sqmi'] = tracts2['total_vandalism'] / tracts2['area_sqmi']
tracts2.plot(column='van_sqmi', cmap='OrRd', figsize=(10,8))

Ah, that looks much better!

### 4.9.3. Interpolate!

If we want to avoid aggregating the data into somewhat arbitrary geographies like census tracts, we can use 2D kernel density estimation to convert our discrete data points into a continuous surface:

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(15,15))
g = blocks_proj.plot(color='none', edgecolor='k', linewidth=.25, ax=ax)
hb = sns.kdeplot(
    x=geovandalism_proj_blocks['geometry'].x, y=geovandalism_proj_blocks['geometry'].y,
    cmap='turbo', fill=True, alpha=0.6)

The code in the next cell does essentially the same thing as the 2D KDE, but instead of estimating a continuous function it estimates a discrete 2D _histogram_ and applies a smoothing function. Code is adapted from [here](http://nbviewer.jupyter.org/gist/perrygeo/c426355e40037c452434)

In [ ]:
from scipy import ndimage
import numpy as np

def heatmap(d, bins=(100,100), smoothing=1.3, cmap='turbo'):
    def getx(pt):
        return pt.coords[0][0]

    def gety(pt):
        return pt.coords[0][1]

    x = list(d.geometry.apply(getx))
    y = list(d.geometry.apply(gety))
    heatmap, xedges, yedges = np.histogram2d(y, x, bins=bins)
    extent = [yedges[0], yedges[-1], xedges[-1], xedges[0]]

    logheatmap = np.log(heatmap)
    logheatmap[np.isneginf(logheatmap)] = 0
    logheatmap = ndimage.gaussian_filter(logheatmap, smoothing, mode='nearest')
    blocks_proj.plot(color='none', edgecolor='white', linewidth=.5, alpha=.5, figsize=(15,10))

    plt.imshow(logheatmap, cmap=cmap, extent=extent)
    plt.colorbar(shrink=.75)
    plt.gca().invert_yaxis()
    plt.show()

heatmap(geovandalism_proj_blocks, bins=190, smoothing=5)

# 5. Exercise: Spatial Ops with BART Stations

In [ ]:
bart = gpd.read_file("http://stacks.stanford.edu/file/druid:sp464ds6332/data.zip")

In [ ]:
bart_proj = bart.to_crs(blocks_proj.crs)
bart_berkeley = gpd.sjoin(bart_proj, blocks_proj, how="inner", op='intersects')
base = blocks_proj.plot(color='white', edgecolor='gray', linewidth=.1)
bart_berkeley.plot(ax=base, color='green', markersize = 20);

In [ ]:
bart_buffer = bart_berkeley['geometry'].buffer(500)
base = blocks_proj.plot(color='none', edgecolor='gray', linewidth=.1)
bart_buffer.plot(ax=base, color='green', alpha=.5);

In [ ]:
bart_buffer_df = gpd.GeoDataFrame(
    {'geometry': bart_buffer, 'bart_buffer_df':['North \n Berkeley','Downtown \n Berley','Ashby']},
    crs = blocks_proj.crs
)
print(bart_buffer_df)

## 5.1. Adding textual info to a map with `ax.annotate()`

Below we find a **representative point** inside each BART station buffer, and use it to position a label centered inside the buffer. We'll use the Station Name as the label.

In [ ]:
bart_buffer_df['coords'] = bart_buffer_df['geometry'].apply(lambda x: x.representative_point().coords[:])
bart_buffer_df['coords'] = [coords[0] for coords in bart_buffer_df['coords']]

In [ ]:
fig, ax = plt.subplots(figsize=(14,10))
base = blocks_proj.plot(color='white', edgecolor='gray', linewidth=.2, ax=ax)
bart_buffer.plot(color='green', markersize = 5, alpha=.5, ax=ax)
for idx, row in bart_buffer_df.iterrows():
    ax.annotate(text=row['bart_buffer_df'], xy=row['coords'],
                 horizontalalignment='center', verticalalignment='center', fontsize=8)

## 5.2. Spatial Overlays

Spatial joins and overlays are very similar. Joins are like merges in Pandas but with GeoDataFrames, so they result in different sets being included in the resulting merge.  Spatial joins are useful for merging point and polygon layers for example, by identifying which points fall into which polygons, and then determining what to do about these relationships based on the operation requested.  

We briefly talked about these last week:

<center><img src="https://geopandas.org/en/stable/_images/overlay_operations.png" width=700></center>

Overlays are more general geoprocessing operations using set theory to produce a geometric result that contains different combinations of the inputs, and are particularly useful if you want to operate on two polygon sets or lines and polygons.

We would import the relevant methods like this:

```python
from geopandas.tools import overlay
```

The `geopandas.tools.overlay()` function takes three arguments:

  * df1
  * df2
  * how

Where `how` can be one of:

 - "intersection"
 - "union"
 - "identity"
 - "symmetric_difference"
 - "difference"

These correspond in set theory to A & B, A or B, A, Not (A or B),  A & (Not B):

| Operation    | Set Equivalent |
|--------------|----------------|
| Intersection | A & B |
| Union        | A or B |
| Identity     | A      |
| Symmetric Difference | Not (A & B) |
| Difference | A & (Not B) |

Let's try these operations out on a modest size dataset (e.g. NOT parcels) so see them in action.

### 5.2.1. Union
Start with union operations.  How does the result change depending on the order of the left and right dataframe?

In [ ]:
gpd.overlay(tracts_proj, bart_buffer_df, how='union').plot(figsize=(10, 8))

### 5.2.2. Identity
OK, how about the identity operator? How does it compare to union?

In [ ]:
gpd.overlay(tracts_proj, bart_buffer_df, how='identity').plot(figsize=(10,6));

In [ ]:
gpd.overlay(bart_buffer_df, tracts_proj, how='identity').plot(figsize=(10,6))

### 5.2.3. Intersection
And how does that result compare to intersection?

In [ ]:
gpd.overlay(tracts_proj, bart_buffer_df, how='intersection').plot();

### 5.2.4. Symmetric Difference
Now compare intersection to symmetric difference.  What is the relationship?

In [ ]:
gpd.overlay(tracts_proj, bart_buffer_df, how='symmetric_difference').plot();

### 5.2.5. Difference
And finally, how does that compare to the difference operator?

In [ ]:
gpd.overlay(tracts_proj, bart_buffer_df, how='difference').plot();

## 5.3. Spatial Join

Census Blocks and BART Station Buffers

In [ ]:
bart_blocks = gpd.sjoin(blocks_proj, bart_buffer_df, how="inner", op='intersects')
bart_blocks.head()

In [ ]:
base = bart_blocks.plot(color='white', edgecolor='black', linewidth=.2, figsize=(10,8))
bart_buffer_df.plot(ax=base, color='green', markersize = 5, alpha=.5)
for idx, row in bart_buffer_df.iterrows():
    plt.annotate(text=row['bart_buffer_df'], xy=row['coords'],
                 horizontalalignment='center', verticalalignment='center', fontsize=12)

## 5.4. Spatial Computations
### 5.4.1. Polygon Centroids

Let's say we want to calculate distance from each census block to the nearest BART station.  If we do it with the polygon geometry of blocks it would be a very expensive calculation - we would have to check for the nearest point on the polygon, and figure out which point on each polygon is closest.  For most purposes that might be more than we need - a simple calculation fom the centroid of the parcel might serve our purposes just as well, and be much much faster to compute. So let's do that.  But wait, we don't have a set of block centroids...  fortunately that isn't hard to get.

In [ ]:
blocks_proj['centroid'] = blocks_proj.centroid
blocks_proj = blocks_proj.set_geometry('centroid')
print(blocks_proj.geometry.name)
blocks_proj.head()

### 5.4.2. Distance Calculations

Below we use a lamda function to compute the nearest distance of each vandalism case to each of the 3 BART stations, storing the distance to the nearest BART station as a new column in the vandalism GeoDataFrame.

In [ ]:
geovandalism_proj['min_dist_to_bart'] = geovandalism_proj.geometry.apply(
    lambda g: bart_berkeley.distance(g).min())

And now we can plot the vandalism points using a color-ramp scaled by the minimum distance to a BART station.

In [ ]:
base = blocks_proj.set_geometry('geometry').plot(color='white', edgecolor='gray', linewidth=.1, figsize=(14,10))
geovandalism_proj.plot(ax=base, column='min_dist_to_bart', cmap='turbo', markersize=3, legend=True)
bart_buffer_df.plot(ax=base, color='green', markersize = 5, alpha=.5)
for idx, row in bart_buffer_df.iterrows():
    plt.annotate(text=row['bart_buffer_df'], xy=row['coords'],
                 horizontalalignment='center', verticalalignment='center', fontsize=8)

We could have done this just as easily to compute the distances from each of our Census Block centroids

In [ ]:
blocks_proj['min_dist_to_bart'] = blocks_proj.geometry.apply(lambda g: bart_berkeley.distance(g).min())

In [ ]:
fig, ax = plt.subplots(figsize=(16,12))
blocks_proj.set_geometry('geometry').plot(ax=ax, column='min_dist_to_bart', cmap='turbo', marker='o', markersize=.5, alpha=.5, legend=True)
bart_buffer_df.plot(ax=ax, color='white', edgecolor='black', markersize = 5, alpha=.75)
for idx, row in bart_buffer_df.iterrows():
    plt.annotate(text=row['bart_buffer_df'], xy=row['coords'],
                 horizontalalignment='center', verticalalignment='center', fontsize=9)

# 6. Your Turn

To practice with Geopandas, experiment with the methods covered so far with data you are interested in working with on your project, or any other data you can find readily from an Open Data Portal like Berkeley, San Francisco, Oakland, New York, or others.

* Download a shapefile containing point data and attributes
* Create a GeoDataFrame
* Set its CRS
* Plot it with color coding of the points based on the values of an attribute
* Download a shapefile containing polygons and attributes
* Create a GeoDataFrame
* Plot a Choroplethic Map
* Change the coordinate procjetion on these from spherical to a projected coordinate system
* Do a spatial join of the point and polygon data
* Aggregate the joined data to summarize it


# 7. For next time
- Python installs: PySAL

# 8. Questions?

# Sources

- This notebook was heavily adapted from previous course material by [Prof. Paul Waddell](https://urbansim.com/people).